In [2]:
!pip install langchain
!pip install sentence-transformers
!pip install transformers
!pip install huggingface_hub
!pip install faiss-cpu
!pip install torch
!pip install numpy

In [3]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install evaluate bert-score

In [7]:
!pip install rouge_score

  Using cached rouge_score-0.1.2-py3-none-any.whl


In [8]:
import langchain
import sentence_transformers
import transformers
import huggingface_hub
import faiss
import torch
import numpy as np

print("Libraries installed successfully!")

Libraries installed successfully!


In [9]:
from tkinter import Tk
from tkinter.filedialog import askopenfilenames

# Ouvrir une fenêtre pour choisir plusieurs fichiers
Tk().withdraw()  # On cache la fenêtre principale
file_paths = askopenfilenames(title="Sélectionnez les documents", filetypes=[("Documents", "*.pdf *.txt *.docx *.csv")])

# Afficher les chemins sélectionnés
print(file_paths)

# Ensuite tu peux charger ces documents dans ton pipeline


('C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/Actualisation-Guide-Risques-Actifs-Prestataires-Virtuels.pdf.coredownload.pdf.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/bulletinopee_49_final_article4.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/CELEX_32023R1114_EN_TXT.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/FATF-Booklet_VA.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/forum_fintech_2020_atelier_g-crypto-actif_et_lcb-ft_acpr.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/indurp15-f.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/June2023-Targeted-Update-VA-VASP.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/r_qt1809f_fr.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/Rapport_61_F.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/RBA-VA-VASPs.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/RFPI-NS-Blockchain-04.pdf', 'C:/Users/MMD/Desktop/IngenieurDS/PI/DocumentsPDF/S2023_1247_Crypto_acti

In [11]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Charger les fichiers sélectionnés
docs = []
for file_path in file_paths:
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        docs.extend(loader.load())
    elif file_path.endswith(".csv"):
        loader = CSVLoader(file_path=file_path)
        docs.extend(loader.load())

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"✅ Loaded and split {len(docs)} documents into {len(chunks)} chunks.")

✅ Loaded and split 736 documents into 5213 chunks.


In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-base-en-v1.5")

# Define LangChain-compatible wrapper
from langchain.embeddings.base import Embeddings

class BGEEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return model.encode(texts, show_progress_bar=True).tolist()

    def embed_query(self, text):
        return model.encode([text])[0]

embedding = BGEEmbeddings()


In [13]:
from langchain.vectorstores import FAISS

# Build the FAISS vector index
vectorstore = FAISS.from_documents(chunks, embedding)

# Save it locally in Colab
vectorstore.save_local("faiss_index")


Batches:   0%|          | 0/163 [00:00<?, ?it/s]

In [14]:
# Fonction de calcul de similarité
def compute_similarity(query, docs):
    query_embedding = model.encode([query])[0]
    doc_embeddings = model.encode([doc.page_content for doc in docs])
    similarities = np.dot(doc_embeddings, query_embedding)
    return similarities

# Fonction pour calculer les métriques d'évaluation
def recall_at_k(similarities, k):
    sorted_similarities = np.argsort(similarities)[::-1]
    return 1 if any(sorted_similarities[:k]) else 0

def precision_at_k(similarities, k):
    sorted_similarities = np.argsort(similarities)[::-1]
    return np.mean(sorted_similarities[:k])

def hit_at_k(similarities, k):
    sorted_similarities = np.argsort(similarities)[::-1]
    return 1 if any(sorted_similarities[:k]) else 0

def mean_reciprocal_rank(similarities):
    sorted_similarities = np.argsort(similarities)[::-1]
    for rank, similarity in enumerate(sorted_similarities, 1):
        if similarity > 0:
            return 1 / rank
    return 0

In [17]:
# 📌 Ce code force la mise à jour du package regex dans Jupyter

!pip install --upgrade regex

# Après la mise à jour, redémarre le kernel du notebook pour appliquer le changement
print("✅ 'regex' updated successfully. Please restart the Jupyter kernel (menu: Kernel > Restart) before running your code again.")


  Using cached regex-2024.11.6-cp38-cp38-win_amd64.whl.metadata (41 kB)
Using cached regex-2024.11.6-cp38-cp38-win_amd64.whl (274 kB)
  Attempting uninstall: regex
    Found existing installation: regex 2020.6.8
    Uninstalling regex-2020.6.8:
      Successfully uninstalled regex-2020.6.8
✅ 'regex' updated successfully. Please restart the Jupyter kernel (menu: Kernel > Restart) before running your code again.


  You can safely remove it manually.


In [15]:
import numpy as np
import os
import gc
from getpass import getpass
from dataclasses import dataclass

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

import evaluate
from bert_score import score as bert_score
from bert_score import score

import faiss
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

# ------------------------------
# Step 1: Hugging Face Authentication
# ------------------------------

def authenticate_huggingface():
    token = getpass("🔐 Enter your Hugging Face API token: ")
    login(token=token)
    print("✅ Logged into Hugging Face")

# ------------------------------
# Step 2: Memory Optimization Setup
# ------------------------------

def optimize_memory():
    print("⚙️ Configuring memory optimizations...")
    torch.cuda.empty_cache()
    gc.collect()
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

def print_gpu_memory():
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        props = torch.cuda.get_device_properties(device)
        total_memory = props.total_memory / 1e9
        allocated = torch.cuda.memory_allocated(device) / 1e9
        free = (props.total_memory - torch.cuda.memory_allocated(device)) / 1e9
        print(f"GPU Memory: {total_memory:.2f} GB total")
        print(f"Allocated: {allocated:.2f} GB")
        print(f"Free: {free:.2f} GB")
    else:
        print("⚠️ No GPU detected.")

# ------------------------------
# Step 3: Load Model and Tokenizer
# ------------------------------

def load_model_and_tokenizer(model_name):
    print(f"🔄 Loading model and tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    print("✅ Model and tokenizer loaded successfully")
    print_gpu_memory()
    return model, tokenizer

# ------------------------------
# Step 4: Define the BGEEmbeddings class
# ------------------------------

class BGEEmbeddings(Embeddings):
    def __init__(self):
        self.model = SentenceTransformer("BAAI/bge-base-en-v1.5")

    def embed_documents(self, texts):
        return self.model.encode(texts, show_progress_bar=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0]

embedding = BGEEmbeddings()

# ------------------------------
# Step 5: Prompt Construction
# ------------------------------

def build_prompt(user_question, context):
    return f"""
<|system|>
You are a senior compliance officer, highly experienced in the Due Diligence process for crypto asset funds.
Use ONLY the provided documentation context below to answer the user's question.
Keep the answer short, structured, professional, and comprehensive.
If the question is unclear or irrelevant, politely ask for clarification.

Context:
{context}
</|system|>

<|user|>
{user_question}
</|user|>

<|assistant|>
"""

# ------------------------------
# Step 6: Similarity Metrics
# ------------------------------

def compute_similarity(query, docs):
    # ⚠️ Placeholder function
    query_embedding = embedding.embed_query(query)  # Utilisation de embed_query
    doc_embeddings = embedding.embed_documents([doc.page_content for doc in docs])  # Utilisation de embed_documents
    similarities = np.dot(doc_embeddings, query_embedding)
    return similarities

def recall_at_k(similarities, k):
    return sum(similarities[:k]) / sum(similarities) if len(similarities) > 0 else 0

def precision_at_k(similarities, k):
    return sum(1 for s in similarities[:k] if s > 0.5) / k if len(similarities) > 0 else 0

def hit_at_k(similarities, k):
    return int(any(s > 0.7 for s in similarities[:k]))

def mean_reciprocal_rank(similarities):
    for i, s in enumerate(similarities):
        if s > 0.7:
            return 1.0 / (i + 1)
    return 0.0

# ------------------------------
# Step 7: Main Execution
# ------------------------------

def main():
    authenticate_huggingface()
    optimize_memory()

    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model, tokenizer = load_model_and_tokenizer(model_name)

    query = input("❓ Enter your due diligence question: ").strip()
    print(f"🔍 Searching relevant context for: '{query}'")

    # Charger le vectorstore depuis le dossier 'faiss_index'
    try:
        # Charger l'index FAISS à partir des fichiers existants
        vectorstore = FAISS.load_local("faiss_index", embedding, allow_dangerous_deserialization=True)
        docs = vectorstore.similarity_search(query, k=3)  # Ajustez 'k' selon vos besoins
    except Exception as e:
        print(f"⚠️ Error with vectorstore: {e}")
        # Fallback à des documents factices si l'index FAISS ne peut pas être chargé
        @dataclass
        class Document:
            page_content: str

        docs = [
            Document("Sample document content 1 about crypto due diligence."),
            Document("Sample document content 2 about crypto compliance."),
            Document("Sample document content 3 about regulatory requirements."),
            Document("Sample document content 4 about KYC procedures."),
        ]

    similarities = compute_similarity(query, docs)
    k = 3
    print(f"Recall@{k}: {recall_at_k(similarities, k):.4f}")
    print(f"Precision@{k}: {precision_at_k(similarities, k):.4f}")
    print(f"Hit@{k}: {hit_at_k(similarities, k):.4f}")
    print(f"MRR: {mean_reciprocal_rank(similarities):.4f}")

    context = "\n\n".join(doc.page_content for doc in docs)
    full_prompt = build_prompt(query, context)
    print("📝 Prompt built successfully.")

    # Generation
    print("🤖 Generating response...")
    inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    if "<|assistant|>" in generated_text:
        response = generated_text.split("<|assistant|>")[-1].strip()
    else:
        response = generated_text.strip()

    # Afficher la réponse générée
    print("\n✅ Answer:")
    print(response)

    # Afficher les sources utilisées
    print("\n📚 Sources used to answer the question:")
    for i, doc in enumerate(docs):
        print(f"{i + 1}. {doc.page_content[:200]}...")  # Affiche un extrait des 200 premiers caractères de chaque source

    # EVALUATION ICI (sans référence)
    print("\n📏 Evaluating generation quality using sources...")
    evaluate_generation_without_reference(response, docs)

    # Save model and tokenizer
    save_dir = "./saved_model"
    os.makedirs(save_dir, exist_ok=True)
    print(f"💾 Saving model and tokenizer to {save_dir}...")
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print("✅ Model and tokenizer saved.")

    # Clean up
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 Cleaned up GPU memory.")



# ------------------------------
# Step 8: Generation Quality Evaluation (without reference)
# ------------------------------

def evaluate_generation_without_reference(prediction, docs):
    print("\n📏 Evaluating generation quality...")

    # Compare prediction to the top documents used
    print("\n📝 Checking consistency with sources:")
    for i, doc in enumerate(docs):
        print(f"Source {i + 1}: {doc.page_content[:200]}...")

    # Simple consistency check: does the generated response mention key concepts from the documents?
    consistency_score = 0
    for doc in docs:
        if doc.page_content.lower() in prediction.lower():  # Very basic check, could be improved
            consistency_score += 1

    consistency_percentage = (consistency_score / len(docs)) * 100
    print(f"\nConsistency with sources: {consistency_percentage:.2f}%")

    # Calculate BERTScore using the locally installed version
    predictions = [prediction] * len(docs)  # Repeat the prediction for each reference
    references = [doc.page_content for doc in docs]  # Using documents as 'references'

    # Use BERTScore metrics
    P, R, F1 = score(predictions, references, lang="en")

    print(f"BERTScore - Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")




if __name__ == "__main__":
    main()


🔐 Enter your Hugging Face API token: ········
✅ Logged into Hugging Face
⚙️ Configuring memory optimizations...
🔄 Loading model and tokenizer: TinyLlama/TinyLlama-1.1B-Chat-v1.0
✅ Model and tokenizer loaded successfully
⚠️ No GPU detected.
❓ Enter your due diligence question:  What specific risks to investor/consumer protection are being considered in the development of this digital asset regulation?
🔍 Searching relevant context for: 'What specific risks to investor/consumer protection are being considered in the development of this digital asset regulation?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Recall@3: 1.0000
Precision@3: 1.0000
Hit@3: 1.0000
MRR: 1.0000
📝 Prompt built successfully.
🤖 Generating response...

✅ Answer:
The development of this digital asset regulation is being considered to address risks to investor/consumer protection, which includes but is not limited to:

1. Market integrity: The absence of specific rules for crypto-assets outside the scope of consumer protection rules can leave holders of those assets exposed to risks, including in fields not covered by consumer protection rules. The absence of specific rules can also result in substantial risks to market integrity, including in terms of market abuse as well as in terms of financial crime.

2. Financial crime: To address these risks, some Member States have put in place specific rules for all or a subset of crypto-assets that fall outside the scope of consumer protection rules. However, some providers of non-custodial wallets should not fall within the scope of this Regulation.

3. Cybersecurity: Hardware

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.8512, Recall: 0.8847, F1: 0.8675
💾 Saving model and tokenizer to ./saved_model...
✅ Model and tokenizer saved.
🧹 Cleaned up GPU memory.
